# scrape topic index


In [10]:
%pip install chromadb
%pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Linus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Linus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import requests
import time
import chromadb
from bs4 import BeautifulSoup

In [2]:
#load wikipedia article titles

wiki_titles = []
with open("./data/enwiki-latest-all-titles.txt", "r", encoding="utf-8") as f:
    for line in f:
        wiki_titles.append(line.strip())

print(len(wiki_titles))

17436825


In [8]:
Client = chromadb.PersistentClient("./data/chromadb")

In [28]:
#delete collection "wiki_titles"

Client.delete_collection("wiki_titles")

In [27]:
#add wikipedia article titles to chromadb

collection = Client.get_or_create_collection("wiki_titles")

batch_size = 500
num_batches = len(wiki_titles) // batch_size
for i in range(0, len(wiki_titles), batch_size):
    t_start = time.time()
    
    titles=wiki_titles[i:i+batch_size]
    indices = list(range(i, i+len(titles)))
    indices = [str(x) for x in indices]
    collection.add(
        documents=titles,
        ids=indices
    )
    
    time_elapsed = time.time() - t_start
    remaining_time = time_elapsed * (num_batches - i // batch_size)
    
    elements_per_second = batch_size / time_elapsed
    
    if remaining_time > 3600 * 24:
        time_str = f"{remaining_time / 3600 / 24:.1f} days"
    elif remaining_time > 3600:
        time_str = f"{remaining_time / 3600:.1f} hours"
    elif remaining_time > 60:
        time_str = f"{remaining_time / 60:.1f} minutes"
    else:
        time_str = f"{remaining_time:.1f} seconds"
    
    print(f"Batch {i // batch_size + 1} / {num_batches} ({elements_per_second:.1f} elements/s, {time_str} remaining)")

Batch 1 / 34873 (23.5 elements/s, 8.6 days remaining)
Batch 2 / 34873 (23.9 elements/s, 8.4 days remaining)
Batch 3 / 34873 (23.6 elements/s, 8.5 days remaining)
Batch 4 / 34873 (24.6 elements/s, 8.2 days remaining)
Batch 5 / 34873 (24.4 elements/s, 8.3 days remaining)
Batch 6 / 34873 (24.3 elements/s, 8.3 days remaining)
Batch 7 / 34873 (24.3 elements/s, 8.3 days remaining)
Batch 8 / 34873 (24.5 elements/s, 8.2 days remaining)
Batch 9 / 34873 (25.4 elements/s, 7.9 days remaining)
Batch 10 / 34873 (24.8 elements/s, 8.1 days remaining)
Batch 11 / 34873 (24.9 elements/s, 8.1 days remaining)
Batch 12 / 34873 (24.8 elements/s, 8.1 days remaining)
Batch 13 / 34873 (25.2 elements/s, 8.0 days remaining)
Batch 14 / 34873 (25.0 elements/s, 8.1 days remaining)
Batch 15 / 34873 (25.1 elements/s, 8.0 days remaining)
Batch 16 / 34873 (24.9 elements/s, 8.1 days remaining)
Batch 17 / 34873 (24.9 elements/s, 8.1 days remaining)
Batch 18 / 34873 (24.9 elements/s, 8.1 days remaining)
Batch 19 / 34873 (2

KeyboardInterrupt: 

In [ ]:
def query(query: str, num_results: int = 10):
    collection = Client.get_collection("wiki_titles")
    
    t_start = time.time()
    
    r = collection.query(
        query_texts=[query],
        n_results=num_results
    )
    
    return r